# 8.3 Reshaping and Pivoting（整形和旋转）

> 8.1中的操作名
- join：连接
- combine：合并
- reshape：整形

> 8.2中的操作名
> - merge：归并
- concatenate：串联

> 8.3中的操作名
- pivot：旋转
- stack：堆叠

> 我在整个第八章对这些翻译做了更新，其他章节可能没有统一，如果有发现到不统一的翻译，可以在issue里提出，也可以直接pull request


有很多用于整理表格型数据的基本操作，指的就是reshape和pivot。

# 1 Reshaping with Hierarchical Indexing（对多层级索引进行整形）

多层级索引提供一套统一的方法来整理DataFrame中数据。主要有两个操作：

stack
- 这个操作会把列旋转为行

unstack
- 这个会把行变为列

下面我们会给出一些例子。这里有一个DataFrame，我们用字符串数组来作为行和列的索引：

In [2]:
import numpy as np
import pandas as pd

In [3]:
data = pd.DataFrame(np.arange(6).reshape((2, 3)),
                    index=pd.Index(['Ohio', 'Colorado'], name='state'), 
                    columns=pd.Index(['one', 'two', 'three'], 
                    name='number'))
data

number,one,two,three
state,,,
Ohio,0,1,2
Colorado,3,4,5


使用stack方法会把列数据变为行数据，产生一个Series：

In [6]:
result = data.stack()
result

state     number
Ohio      one       0
          two       1
          three     2
Colorado  one       3
          two       4
          three     5
dtype: int32

对于一个有多层级索引的Series，可以用unstack把它变回DataFrame：

In [7]:
result.unstack()

number,one,two,three
state,,,
Ohio,0,1,2
Colorado,3,4,5


默认会把最内层的层级unstack（取消堆叠），stack默认也是这样。我们可以传入一个表示层级的数字或名字，来指定取消堆叠某个层级：

In [18]:
result.unstack(0)

state,Ohio,Colorado
number,,
one,0,3
two,1,4
three,2,5


In [7]:
result.unstack('state')

state,Ohio,Colorado
number,,
one,0,3
two,1,4
three,2,5


如果某个层级里的值不能在subgroup(子组)里找到的话，unstack可能会引入缺失值：

In [19]:
s1 = pd.Series([0, 1, 2, 3], index=['a', 'b', 'c', 'd'])

s2 = pd.Series([4, 5, 6], index=['c', 'd', 'e'])

data2 = pd.concat([s1, s2], keys=['one', 'two'])

In [20]:
data2

one  a    0
     b    1
     c    2
     d    3
two  c    4
     d    5
     e    6
dtype: int64

In [25]:
data2.unstack()

,a,b,c,d,e
one,0.0,1.0,2.0,3.0,NaN
two,NaN,NaN,4.0,5.0,6.0


stack默认会把缺失值过滤掉，所以这两种操作是可逆的：

In [26]:
data2.unstack()

,a,b,c,d,e
one,0.0,1.0,2.0,3.0,NaN
two,NaN,NaN,4.0,5.0,6.0


In [27]:
data2.unstack().stack()

one  a    0.0
     b    1.0
     c    2.0
     d    3.0
two  c    4.0
     d    5.0
     e    6.0
dtype: float64

In [30]:
data2.unstack().stack(dropna=False)

one  a    0.0
     b    1.0
     c    2.0
     d    3.0
     e    NaN
two  a    NaN
     b    NaN
     c    4.0
     d    5.0
     e    6.0
dtype: float64

如果对一个DataFrame使用unstack，被取消堆叠（unstack）的层级会变为结果中最低的层级：

In [34]:
result

state     number
Ohio      one       0
          two       1
          three     2
Colorado  one       3
          two       4
          three     5
dtype: int32

In [42]:
df = pd.DataFrame({'left':result,'right':result+5}, 
                 )
df # 行的话，有state和number两个层级，number是内层级。而列的话有side这一个层级

left  right
state    number             
Ohio     one        0      5
         two        1      6
         three      2      7
Colorado one        3      8
         two        4      9
         three      5     10

In [45]:
df.unstack('state')  # state被unstack后，变为比side更低的层级

left          right         
state  Ohio Colorado  Ohio Colorado
number                             
one       0        3     5        8
two       1        4     6        9
three     2        5     7       10

调用stack的时候，可以指明想要stack（堆叠）哪一个轴：

In [52]:
df.columns.name = 'side'
df

side             left  right
state    number             
Ohio     one        0      5
         two        1      6
         three      2      7
Colorado one        3      8
         two        4      9
         three      5     10

# 2 Pivoting “Long” to “Wide” Format（把“长”格式旋转为“宽”格式）

一种用来把多重时间序列数据存储在数据库和CSV中的格式叫long or stacked format（长格式或堆叠格式）。下面我们加载一些数据，处理一下时间序列文件并做一些数据清理工作：

In [53]:
data = pd.read_csv('../examples/macrodata.csv')
data.head()

,year,quarter,realgdp,realcons,realinv,realgovt,realdpi,cpi,m1,tbilrate,unemp,pop,infl,realint
0,1959.0,1.0,2710.349,1707.4,286.898,470.045,1886.9,28.98,139.7,2.82,5.8,177.146,0.00,0.00
1,1959.0,2.0,2778.801,1733.7,310.859,481.301,1919.7,29.15,141.7,3.08,5.1,177.830,2.34,0.74
2,1959.0,3.0,2775.488,1751.8,289.226,491.260,1916.4,29.35,140.5,3.82,5.3,178.657,2.74,1.09
3,1959.0,4.0,2785.204,1753.7,299.356,484.052,1931.3,29.37,140.0,4.33,5.6,179.386,0.27,4.06
4,1960.0,1.0,2847.699,1770.5,331.722,462.199,1955.5,29.54,139.6,3.50,5.2,180.007,2.31,1.19


In [57]:
periods = pd.PeriodIndex(year=data.year, quarter=data.quarter)
periods

PeriodIndex(['1959Q1', '1959Q2', '1959Q3', '1959Q4', '1960Q1', '1960Q2',
             '1960Q3', '1960Q4', '1961Q1', '1961Q2',
             ...
             '2007Q2', '2007Q3', '2007Q4', '2008Q1', '2008Q2', '2008Q3',
             '2008Q4', '2009Q1', '2009Q2', '2009Q3'],
            dtype='period[Q-DEC]', length=203, freq='Q-DEC')

In [58]:
pd.PeriodIndex?

Init signature:
pd.PeriodIndex(
    data=None,
    ordinal=None,
    freq=None,
    tz=None,
    dtype=None,
    copy=False,
    name=None,
    **fields,
)
Docstring:     
Immutable ndarray holding ordinal values indicating regular periods in time.

Index keys are boxed to Period objects which carries the metadata (eg,
frequency information).

Parameters
----------
data : array-like (1d int np.ndarray or PeriodArray), optional
    Optional period-like data to construct index with.
copy : bool
    Make a copy of input ndarray.
freq : str or period object, optional
    One of pandas period strings or corresponding objects
year : int, array, or Series, default None
month : int, array, or Series, default None
quarter : int, array, or Series, default None
day : int, array, or Series, default None
hour : int, array, or Series, default None
minute : int, array, or Series, default None
second : int, array, or Series, default None
tz : object, default None
    Timezone for converting datetime64

In [60]:
columns = pd.Index(['realgdp', 'infl', 'unemp'], name='item')
columns

Index(['realgdp', 'infl', 'unemp'], dtype='object', name='item')

In [62]:
data = data.reindex(columns=columns)
data

item,realgdp,infl,unemp
0,2710.349,0.00,5.8
1,2778.801,2.34,5.1
2,2775.488,2.74,5.3
3,2785.204,0.27,5.6
4,2847.699,2.31,5.2
...,...,...,...
198,13324.600,-3.16,6.0
199,13141.920,-8.79,6.9
200,12925.410,0.94,8.1
201,12901.504,3.37,9.2


In [66]:
periods.to_timestamp?

Signature: periods.to_timestamp(*args, **kwargs)
Docstring:
Cast to DatetimeArray/Index.

Parameters
----------
freq : str or DateOffset, optional
    Target frequency. The default is 'D' for week or longer,
    'S' otherwise.
how : {'s', 'e', 'start', 'end'}
    Whether to use the start or end of the time period being converted.

Returns
-------
DatetimeArray/Index
File:      c:\users\qli32\ana\lib\site-packages\pandas\core\accessor.py
Type:      method


In [73]:
data.index = periods.to_timestamp('D', 'start')
data

item,realgdp,infl,unemp
1959-01-01,2710.349,0.00,5.8
1959-04-01,2778.801,2.34,5.1
1959-07-01,2775.488,2.74,5.3
1959-10-01,2785.204,0.27,5.6
1960-01-01,2847.699,2.31,5.2
...,...,...,...
2008-07-01,13324.600,-3.16,6.0
2008-10-01,13141.920,-8.79,6.9
2009-01-01,12925.410,0.94,8.1
2009-04-01,12901.504,3.37,9.2


In [82]:
data.stack().reset_index().rename(columns={0:'value'})

,level_0,item,value
0,1959-01-01,realgdp,2710.349
1,1959-01-01,infl,0.000
2,1959-01-01,unemp,5.800
3,1959-04-01,realgdp,2778.801
4,1959-04-01,infl,2.340
...,...,...,...
604,2009-04-01,infl,3.370
605,2009-04-01,unemp,9.200
606,2009-07-01,realgdp,12990.341
607,2009-07-01,infl,3.560


In [83]:
ldata = data.stack().reset_index().rename(columns={0: 'value'})

对于PeriodIndex，我们会在第十一章讲得更详细些。在这里，这个函数把year和quarter这两列整合起来作为一种时间间隔类型。

ldata看起来是这样的：

In [84]:
ldata[:10]

,level_0,item,value
0,1959-01-01,realgdp,2710.349
1,1959-01-01,infl,0.000
2,1959-01-01,unemp,5.800
3,1959-04-01,realgdp,2778.801
4,1959-04-01,infl,2.340
5,1959-04-01,unemp,5.100
6,1959-07-01,realgdp,2775.488
7,1959-07-01,infl,2.740
8,1959-07-01,unemp,5.300
9,1959-10-01,realgdp,2785.204


这种格式叫做long format for multiple time series（用于多重时间序列的长格式），或者有两个以上键（keys）的观测数据（在这个例子里，keys指的是date和item）。表格中的每一行表示一个观测数据。

这种数据经常被存储于关系型数据库中，比如MySQL，这种固定的模式（列名和数据类型）能让作为item列中不同的数据，添加到表格中。在前一个例子里，date和item通常被用来当做primary keys（主键，这是关系型数据库里的术语），能实现relational integrity（关系完整性）和更方便的join（联结）。但是在一些例子里，这种格式的数据并不好处理；我们可能更喜欢有一个DataFrame，其中一列能有不同的item值，并用date列作为索引。DataFrame中的pivot方法，就能做到这种转换：

In [92]:
ldata = ldata.rename(columns={'level_0':'date'})
ldata.index.name = ''
ldata

,date,item,value
,,,
0,1959-01-01,realgdp,2710.349
1,1959-01-01,infl,0.000
2,1959-01-01,unemp,5.800
3,1959-04-01,realgdp,2778.801
4,1959-04-01,infl,2.340
...,...,...,...
604,2009-04-01,infl,3.370
605,2009-04-01,unemp,9.200
606,2009-07-01,realgdp,12990.341


In [93]:
ldata.pivot(index='date',columns='item',values='value')

item,infl,realgdp,unemp
date,,,
1959-01-01,0.00,2710.349,5.8
1959-04-01,2.34,2778.801,5.1
1959-07-01,2.74,2775.488,5.3
1959-10-01,0.27,2785.204,5.6
1960-01-01,2.31,2847.699,5.2
...,...,...,...
2008-07-01,-3.16,13324.600,6.0
2008-10-01,-8.79,13141.920,6.9
2009-01-01,0.94,12925.410,8.1


In [94]:
pivoted = ldata.pivot('date', 'item', 'value')
pivoted

item,infl,realgdp,unemp
date,,,
1959-01-01,0.00,2710.349,5.8
1959-04-01,2.34,2778.801,5.1
1959-07-01,2.74,2775.488,5.3
1959-10-01,0.27,2785.204,5.6
1960-01-01,2.31,2847.699,5.2
...,...,...,...
2008-07-01,-3.16,13324.600,6.0
2008-10-01,-8.79,13141.920,6.9
2009-01-01,0.94,12925.410,8.1


前两个传入的值是列，分别被用于作为行索引和列索引（date是行索引，item是列索引），最后是一个是可选的value column（值列），用于填充DataFrame。假设我们有两列值，我们想要同时整形：

In [95]:
ldata['value2'] = np.random.randn(len(ldata))
ldata[:10]

,date,item,value,value2
,,,,
0,1959-01-01,realgdp,2710.349,2.425422
1,1959-01-01,infl,0.000,1.004718
2,1959-01-01,unemp,5.800,1.396915
3,1959-04-01,realgdp,2778.801,-0.816046
4,1959-04-01,infl,2.340,-0.188584
5,1959-04-01,unemp,5.100,2.096094
6,1959-07-01,realgdp,2775.488,-1.825862
7,1959-07-01,infl,2.740,0.991839
8,1959-07-01,unemp,5.300,0.255127


舍弃最后一个参数，我们能得到一个有多层级列的DataFrame：

In [97]:
pivoted = ldata.pivot('date', 'item', values=['value','value2'])
pivoted[:5]

value                    value2                    
item        infl   realgdp unemp      infl   realgdp     unemp
date                                                          
1959-01-01  0.00  2710.349   5.8  1.004718  2.425422  1.396915
1959-04-01  2.34  2778.801   5.1 -0.188584 -0.816046  2.096094
1959-07-01  2.74  2775.488   5.3  0.991839 -1.825862  0.255127
1959-10-01  0.27  2785.204   5.6 -1.611401 -0.914629  1.105922
1960-01-01  2.31  2847.699   5.2  0.379821 -0.028575 -2.593810

In [98]:
pivoted['value'][:5]

item,infl,realgdp,unemp
date,,,
1959-01-01,0.00,2710.349,5.8
1959-04-01,2.34,2778.801,5.1
1959-07-01,2.74,2775.488,5.3
1959-10-01,0.27,2785.204,5.6
1960-01-01,2.31,2847.699,5.2


这里pivot相当于用set_index创建了一个多层级用里，并调用了unstack：

In [104]:
unstacked = ldata.set_index(['date', 'item']).unstack('item')
unstacked[:7]

value                    value2                    
item        infl   realgdp unemp      infl   realgdp     unemp
date                                                          
1959-01-01  0.00  2710.349   5.8  1.004718  2.425422  1.396915
1959-04-01  2.34  2778.801   5.1 -0.188584 -0.816046  2.096094
1959-07-01  2.74  2775.488   5.3  0.991839 -1.825862  0.255127
1959-10-01  0.27  2785.204   5.6 -1.611401 -0.914629  1.105922
1960-01-01  2.31  2847.699   5.2  0.379821 -0.028575 -2.593810
1960-04-01  0.14  2834.390   5.2 -0.144735  1.165998  0.860801
1960-07-01  2.70  2839.022   5.6 -0.912608  0.472015 -1.533155

# 3 Pivoting “Wide” to “Long” Format（把“宽”格式旋转为“长”格式）

用于DataFrame，与pivot相反的操作是pandas.melt。相对于把一列变为多列的pivot，melt会把多列变为一列，产生一个比输入的DataFrame还要长的结果。看一下例子：

In [106]:
df = pd.DataFrame({'key': ['foo', 'bar', 'baz'], 
                   'A': [1, 2, 3], 
                   'B': [4, 5, 6], 
                   'C': [7, 8, 9]})
df

,key,A,B,C
0,foo,1,4,7
1,bar,2,5,8
2,baz,3,6,9


'key'列可以作为group indicator（群指示器），其他列可以作为数据值。当使用pandas.melt，我们必须指明哪些列是群指示器。这里我们令key作为群指示器：

In [113]:
df.set_index(['key']).stack().reset_index().sort_values('level_1')

,key,level_1,0
0,foo,A,1
3,bar,A,2
6,baz,A,3
1,foo,B,4
4,bar,B,5
7,baz,B,6
2,foo,C,7
5,bar,C,8
8,baz,C,9


In [114]:
melted = pd.melt(df, ['key'])
melted

,key,variable,value
0,foo,A,1
1,bar,A,2
2,baz,A,3
3,foo,B,4
4,bar,B,5
5,baz,B,6
6,foo,C,7
7,bar,C,8
8,baz,C,9


使用pivot，我们可以得到原来的布局：

In [120]:
reshaped = melted.pivot('key', 'variable', 'value')
reshaped

variable,A,B,C
key,,,
bar,2,5,8
baz,3,6,9
foo,1,4,7


因为pivot会给行标签创建一个索引（key列），所以这里我们要用reset_index来让数据变回去：

In [137]:
reshaped.reset_index()

variable,key,A,B,C
0,bar,2,5,8
1,baz,3,6,9
2,foo,1,4,7


当然，我们也可以在使用melt的时候指定哪些列用于值：

In [34]:
pd.melt(df, id_vars=['key'], value_vars=['A', 'B'])

,key,variable,value
0,foo,A,1
1,bar,A,2
2,baz,A,3
3,foo,B,4
4,bar,B,5
5,baz,B,6


pandas.melt也能在没有群指示器的情况下使用：

In [125]:
pd.melt(df, ['key'])

,key,variable,value
0,foo,A,1
1,bar,A,2
2,baz,A,3
3,foo,B,4
4,bar,B,5
5,baz,B,6
6,foo,C,7
7,bar,C,8
8,baz,C,9


In [123]:
pd.melt(df, value_vars=['A', 'B', 'C'])

,variable,value
0,A,1
1,A,2
2,A,3
3,B,4
4,B,5
5,B,6
6,C,7
7,C,8
8,C,9


In [135]:
pd.melt(df, value_vars=['A', 'B'], id_vars=['key']).pivot(index='key',columns='variable',values='value')

variable,A,B
key,,
bar,2,5
baz,3,6
foo,1,4
